# Project Directory

In [ ]:
projectDir = '/projects/f_mc1689_1/ReliableFC'
scriptsDir = f'{projectDir}/docs/scripts'

# Diffusion MRI

## Diffusion Preprocessing
- List the subjects in subjectsToProcess.tsv (by running this code block)
- Connect to Amarel-Piscataway and run diffusionPrepro_batch.sh

In [1]:
import pandas as pd
import trackProcessing

preproFile = f'{scriptsDir}/diffusion/subjectsToProcess.tsv'

subjectsFile = f'{scriptsDir}/subjects.tsv'
subjectsDF = pd.read_csv(subjectsFile,sep='\t',index_col='subjects')
subjList = subjectsDF.index.values

trackProcessing.update()
subjList = trackProcessing.getSubjects('diffusion preprocessing',['data downloaded'])
print(subjList)
print(len(subjList))

preproDF = pd.DataFrame({'subjects': subjList})
preproDF.to_csv(path_or_buf=preproFile, sep=',', header=False, index=False, index_label=False)

0


## Diffusion Tractography using DSI Studio

In [2]:
import os
import trackProcessing

trackProcessing.update()
subjList = trackProcessing.getSubjects('tractography',['diffusion preprocessing'])
print(subjList)
print(len(subjList))

for subj in subjList:
    os.system(f'{scriptsDir}/diffusion/DSIStudio_batch.sh {subj})'


1
Submitted batch job 2970144


# Functional MRI

## Downsample to Parcels

In [1]:
import os
import trackProcessing
from postprocessing.CIFTIdownsample import CIFTIdownsample

trackProcessing.update()
subjList = trackProcessing.getSubjects('cifti parcellation',['data downloaded'])
print(subjListProc)
print(len(subjListProc))

CIFTIdownsample(subjListProc)

[]
0


## Brain Segmentation Masks for Nuisance Regression

In [2]:
import os
import trackProcessing

trackProcessing.update()
subjList = trackProcessing.getSubjects('brain masks',['data downloaded'])
print(subjList)
print(len(subjList))

for subj in subjList:
    command = f'{scriptsDir}/postprocessing/createMasks_batch.sh {subj}'
    os.system(command)

0


## Nuisance Regression

In [3]:
import os
import trackProcessing
import sys
sys.path.append(f'{scriptsDir}/postprocessing')
import nuisanceRegression

#trackProcessing.update()
subjList = trackProcessing.getSubjects('nuisance regressors',['cifti parcellation','brain masks'])
#print(subjList)
print(len(subjList))

runsList = ['rfMRI_REST1_PA','rfMRI_REST1_AP','rfMRI_REST2_PA','rfMRI_REST2_AP','tfMRI_CARIT_PA']

nuisanceRegression.step1_createNuisanceRegressors(subjList,runsList=runsList,nproc=1)

5
creating nuisance regressors for subject HCA6102038_V1_MR run: rfMRI_REST1_AP
Loading raw fMRI data
Obtaining standard global, wm, and ventricle signals and their derivatives
Obtaining aCompCor regressors and their derivatives
creating nuisance regressors for subject HCA6102038_V1_MR run: rfMRI_REST2_AP
Loading raw fMRI data
Obtaining standard global, wm, and ventricle signals and their derivatives
Obtaining aCompCor regressors and their derivatives
creating nuisance regressors for subject HCA6102038_V1_MR run: rfMRI_REST1_PA
Loading raw fMRI data
Obtaining standard global, wm, and ventricle signals and their derivatives
Obtaining aCompCor regressors and their derivatives
creating nuisance regressors for subject HCA6102038_V1_MR run: rfMRI_REST2_PA
Loading raw fMRI data
Obtaining standard global, wm, and ventricle signals and their derivatives
Obtaining aCompCor regressors and their derivatives
creating nuisance regressors for subject HCA6470673_V1_MR run: rfMRI_REST1_AP
Loading raw 

In [1]:
import os
import trackProcessing
import sys
sys.path.append(f'{scriptsDir}/postprocessing')
import nuisanceRegression

#trackProcessing.update()
#subjList = trackProcessing.getSubjects('nuisance regression',['nuisance regressors'])

import pandas as pd
subjectsFile = f'/projects/f_mc1689_1/ReliableFC/docs/scripts/subjects.tsv'
subjectsDF = pd.read_csv(subjectsFile,sep='\t',index_col='subjects')
subjList = subjectsDF.index.values
#print(subjList)
print(len(subjList))

runsList = ['rfMRI_REST1_PA','rfMRI_REST1_AP','rfMRI_REST2_PA','rfMRI_REST2_AP']

nuisanceRegression.step2_nuisanceRegression(subjList,runsList=runsList,nproc=8,model='24pXaCompCorXVolterra',spikeReg=False,zscore=False)


236
Running regression on subject HCA6002236_V1_MR | run rfMRI_REST1_AP
	Nuisance Model: 36p with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA6002236_V1_MR | run rfMRI_REST2_AP
	Nuisance Model: 36p with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA6002236_V1_MR | run rfMRI_REST1_PA
	Nuisance Model: 36p with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA6002236_V1_MR | run rfMRI_REST2_PA
	Nuisance Model: 36p with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA6030645_V1_MR | run rfMRI_REST1_AP
	Nuisance Model: 36p with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA6030645_V1_MR | run rfMRI_REST2_AP
	Nuisance Model: 36p with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA6030645_V1_MR | run rfMRI_REST1_PA
	Nuisance Model: 36p with spikeReg: False | zscore: False
	Task 

In [3]:
import pandas as pd
import os
import h5py
import sys
sys.path.append(f'{scriptsDir}/postprocessing')
import nuisanceRegression
import CIFTIdownsample

dataDir = f'{projectDir}/data'

subjectsFile = f'{scriptsDir}/subjects.tsv'
subjectsDF = pd.read_csv(subjectsFile,sep='\t',index_col='subjects')
subjList = subjectsDF.index.values

runsList = ['rfMRI_REST1_PA','rfMRI_REST1_AP','rfMRI_REST2_PA','rfMRI_REST2_AP']

for s,subj in enumerate(subjList):
    missing = False
    h5filename = f'{dataDir}/preprocessed/postproc/{subj}_V1_MR_glmOutput_cortexsubcortex_data.h5'
    if not os.path.isfile(h5filename):
        missing = True
    else:
        with h5py.File(h5filename,'r') as h5file:
            try:
                for run in runsList:
                    data = h5file[f'{run}/nuisanceReg_resid_24pXaCompCorXVolterra'][:]
            except:
                missing = True
                
    if missing:
        print(s,subj)
        if not os.path.isfile(f'{dataDir}/preprocessed/nuisanceRegressors/{subj}_V1_MR_nuisanceRegressors.h5'):
            if not os.path.isfile(f'{dataDir}/preprocessed/{subj}_V1_MR/MNINonLinear/Results/{runsList[0]}/{runsList[0]}_Atlas_MSMAll_cab-np.ptseries.nii'):
                CIFTIdownsample.CIFTIdownsample([subj])
            if not os.path.isfile(f'{dataDir}/preprocessed/{subj}_V1_MR/masks/{subj}_V1_MR_ventricles_func_eroded.nii.gz'):
                os.system(f'{scriptsDir}/postprocessing/createMasks_batch.sh {subj}')
                continue
            nuisanceRegression.step1_createNuisanceRegressors([subj],nproc=1)
        try:
            nuisanceRegression.step2_nuisanceRegression([subj],nproc=1,model='24pXaCompCorXVolterra',vertices=False)
        except:
            nuisanceRegression.step1_createNuisanceRegressors([subj],nproc=1)
            nuisanceRegression.step2_nuisanceRegression([subj],nproc=1,model='24pXaCompCorXVolterra',vertices=False)
    

278 HCA9938309
Running regression on subject HCA9938309_V1_MR | run rfMRI_REST1_PA
	Nuisance Model: 24pXaCompCorXVolterra with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA9938309_V1_MR | run rfMRI_REST1_AP
	Nuisance Model: 24pXaCompCorXVolterra with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA9938309_V1_MR | run rfMRI_REST2_PA
	Nuisance Model: 24pXaCompCorXVolterra with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA9938309_V1_MR | run rfMRI_REST2_AP
	Nuisance Model: 24pXaCompCorXVolterra with spikeReg: False | zscore: False
	Task Model: None
278 HCA9938309
Running regression on subject HCA9938309_V1_MR | run rfMRI_REST1_PA
	Nuisance Model: 24pXaCompCorXVolterra with spikeReg: False | zscore: False
	Task Model: None
Running regression on subject HCA9938309_V1_MR | run rfMRI_REST1_AP
	Nuisance Model: 24pXaCompCorXVolterra with spikeReg: False | zscore: False
	Task Model: Non

## CARIT Task Regression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import h5py
import nibabel as nib
from nipy.modalities.fmri.hemodynamic_models import spm_hrf

# Project directories
dataDir = '/projects/f_mc1689_1/ReliableFC/data'
scriptsDir = '/projects/f_mc1689_1/ReliableFC/docs/scripts'

# Subject numbers
#dataset = 'discovery'
subjectsFile = f'{scriptsDir}/subjects.tsv'
subjectsDF = pd.read_csv(subjectsFile,sep='\t',index_col='subjects')
subjList = subjectsDF.index.values
nSubjs = len(subjList)

# Run name
runsList = ['tfMRI_CARIT_PA']
run = 'tfMRI_CARIT_PA'

# fMRI acquisition
TR = 0.800
nTRs = 290 # after preprocessing

In [ ]:
def HRF_from_onsets(onsets,durations,TR=TR,nTRs=nTRs,upsample_rate=0.01):
    '''
    onsets = onsets in seconds
    durations = onset length in seconds
    nTRs = the total number of images in the scan
    TR = the TR in seconds
    upsample_rate = the upsampled units in seconds

    returns a TR length convolved vector
    '''
    # set up HRF
    HRF = spm_hrf(upsample_rate,1)

    # vector of onsets (1's and 0's)
    onset_vector_up = np.zeros((int(nTRs / (upsample_rate / TR))))
    
    # use the middle of the TR for downsampling
    TR_mid = int((TR / upsample_rate)/2)

    #upsample onsets and durations
    onsets_up = onsets / upsample_rate
    durations_up = durations / upsample_rate
    
    if onsets_up.size == 1:
        # onsets and offsets in upsample space
        on = np.round(onsets_up)
        off = np.round(onsets_up + durations_up)
        
        # add to onsets vector in TR space
        onset_vector_up[np.arange(on,off,1,dtype=int)] = 1
        
    else:
        for trial in range(onsets_up.size):
            # onsets and offsets in upsample space
            on = np.round(onsets_up[trial])
            off = np.round(onsets_up[trial] + durations_up[trial])
            
            # add to onsets vector in TR space
            onset_vector_up[np.arange(on,off,1,dtype=int)] = 1
    
    # convolve HRF with binary onsets
    conv_onset_up = np.convolve(onset_vector_up,HRF)

    # shorten to length of scan
    conv_onset_up = conv_onset_up[0:len(onset_vector_up)]

    # downsample back to TR space by slicing convolved timeseries
    output = conv_onset_up[TR_mid:len(conv_onset_up):int(TR / upsample_rate)]
    output = output[:,np.newaxis]
    print(output.shape)

    return output

In [ ]:
taskRegModel = 'canonicalHRF'

# Loop through subjects   
for subj in subjList:
    # Open file with detailed trial info - mostly 'A' but some subjects have 'B'
    try:
        caritFile = f'{dataDir}/behavior/CARIT/CARIT_{subj}_V1_A_run1_wide.csv'
        caritDF = pd.read_csv(caritFile)
    except:
        caritFile = f'{dataDir}/behavior/CARIT/CARIT_{subj}_V1_B_run1_wide.csv'
        caritDF = pd.read_csv(caritFile)
    
    # Make separate column with reaction time 
    caritDF['RT'] = caritDF['trialResp.firstRt'] - caritDF['shapeStartTime']
    
    if (taskRegModel=='canonicalHRF'):
        # For trials with impossibly fast responses (< 250 ms), set response as incorrect 
        caritDF.loc[caritDF['RT']<=0.25, 'corrRespCode'] = 0
        caritDF.loc[caritDF['RT']<=0.25, 'RT'] = np.nan
    
        # Get event onsets and durations, make canonical HRF regressor for each condition
        # (hit, miss, correct rejection, and false alarm)
        hits = (caritDF['corrAns']=='go') & (caritDF['corrRespCode']==1)
        hitOnsets = caritDF.loc[hits, 'shapeStartTime'].values - 8
        hitDurations = caritDF.loc[hits, 'shapeEndTime'].values - caritDF.loc[hits, 'shapeStartTime'].values
    
        misses = (caritDF['corrAns']=='go') & (caritDF['corrRespCode']==0)
        missOnsets = caritDF.loc[misses, 'shapeStartTime'].values - 8
        missDurations = caritDF.loc[misses, 'shapeEndTime'].values - caritDF.loc[misses, 'shapeStartTime'].values
    
        CRs = (caritDF['corrAns']=='nogo') & (caritDF['corrRespCode']==1)
        crOnsets = caritDF.loc[CRs, 'shapeStartTime'].values - 8
        crDurations = caritDF.loc[CRs, 'shapeEndTime'].values - caritDF.loc[CRs, 'shapeStartTime'].values
    
        FAs = (caritDF['corrAns']=='nogo') & (caritDF['corrRespCode']==0)
        faOnsets = caritDF.loc[FAs, 'shapeStartTime'].values - 8
        faDurations = caritDF.loc[FAs, 'shapeEndTime'].values - caritDF.loc[FAs, 'shapeStartTime'].values
        
        if taskRegModel=='canonicalHRF':
            hitRegressors = HRF_from_onsets(hitOnsets,hitDurations)
            missRegressors = HRF_from_onsets(missOnsets,missDurations)
            crRegressors = HRF_from_onsets(crOnsets,crDurations)
            faRegressors = HRF_from_onsets(faOnsets,faDurations)
            taskRegressors = np.hstack((hitRegressors,missRegressors,crRegressors,faRegressors))
    
    # Save regressors to h5 data file
    h5filename = f'{dataDir}/preprocessed/postproc/{subj}_V1_MR_glmOutput_cortexsubcortex_data.h5'
    with h5py.File(h5filename,'a') as h5file:
        outname = f'tfMRI_CARIT_PA/{taskRegModel}/taskRegressors'
        try:
            h5file.create_dataset(outname,data=taskRegressors)
        except:
            del h5file[outname] 
            h5file.create_dataset(outname,data=taskRegressors)
    

In [ ]:
sys.path.append(f'{scriptsDir}/postprocessing')
import nuisanceRegression

runsList = ['tfMRI_CARIT_PA']
taskRegModel = 'canonicalHRF'

for subj in subjList:
    nuisanceRegression.step2_nuisanceRegression([subj],runsList=runsList,nproc=1,model='24pXaCompCorXVolterra',spikeReg=False,zscore=False,taskRegModel=taskRegModel,vertices=False)


In [ ]:
# Subject numbers
dataset = 'discovery'
subjectsFile = f'{scriptsDir}/subjects_{dataset}.tsv'
subjectsDF = pd.read_csv(subjectsFile,sep='\t',index_col='subjects')
subjList = subjectsDF.index.values
nSubjs = len(subjList)

# Order of parcels by network
parcType = 'cortex'
parcOrderFile = f'{scriptsDir}/cabnp/{parcType}_community_order.txt'
parcOrder = pd.read_csv(parcOrderFile, sep='\t', header=None)[0] - 1
parcOrderReverse = parcOrder.sort_values().index
nNodes = len(parcOrder)

taskAct = np.full((nNodes,4,nSubjs),np.nan)

for s,subj in enumerate(subjList):
    h5filename = f'{dataDir}/preprocessed/postproc/{subj}_V1_MR_glmOutput_cortexsubcortex_data.h5'
    with h5py.File(h5filename,'r') as h5file:
        outname = f'tfMRI_CARIT_PA/{taskRegModel}/taskBetas_24pXaCompCorXVolterra'
        try:
            taskAct[:,:,s] = h5file[outname][:][parcOrder]
        except:
            print(subj)

np.save(f'{dataDir}/postproc/taskActivations_{parcType}_carit_{dataset}.npy',taskAct)
np.save(f'{dataDir}/postproc/taskActivations_{parcType}_carit2_{dataset}.npy',taskAct[:,[0,2]])